# 调用手机摄像头，逐帧实时拍摄+OpenCV图像处理

同济子豪兄 2023-6-25

## 导入工具包

In [1]:
import time

import cv2

from cvs import *

## 初始化摄像头

In [2]:
# 摄像头ID 0-后置 1-前置
Camera_ID = 0

In [3]:
cap = cvs.VideoCapture(Camera_ID)

remi.server      INFO     Started httpserver http://0.0.0.0:60194/


('app runs on port:', 60194)
open the cam:0 ...


## 逐帧处理函数（选择一个运行）

逐帧处理函数，默认不进行任何处理，直接将摄像头捕获的画面写入视频帧

In [4]:
# 什么都不做
def process_frame(img_bgr):
    '''输入BGR格式的 numpy array，输出BGR格式的 numpy array'''
    # img_bgr = img_bgr[:,:,::-1] # BGR转RGB
    return img_bgr

In [7]:
# Canny 边缘检测
def process_frame(img_bgr):
    '''输入BGR格式的 numpy array，输出BGR格式的 numpy array'''
    img_bgr = cv2.Canny(img_bgr, 100, 200)
    img_bgr = np.dstack((img_bgr, img_bgr, img_bgr))
    return img_bgr

remi.server.ws   INFO     connection established: ('127.0.0.1', 37590)
remi.server.ws   INFO     handshake complete


In [4]:
# Canny 边缘检测 - 写 FPS 数值
def process_frame(img_bgr):
    '''输入BGR格式的 numpy array，输出BGR格式的 numpy array'''
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    # 逐帧处理操作
    img_bgr = cv2.Canny(img_bgr, 100, 200)
    img_bgr = np.dstack((img_bgr, img_bgr, img_bgr))
    
    # 记录该帧处理完毕的时间
    end_time = time.time()
    # 计算每秒处理图像帧数FPS
    FPS = 1/(end_time - start_time)

    # 在画面上写 FPS 数值
    end_time = time.time()
    FPS = 1/(end_time - start_time) # 计算每秒处理图像帧数FPS
    FPS_string = 'FPS {:.2f}'.format(FPS) # 写在画面上的字符串
    img_bgr = cv2.putText(img_bgr, FPS_string, (25, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 0, 255), 2) # 在画面上写字：图片，字符串，左上角坐标，字体，字体大小，颜色，字体粗细

    return img_bgr

In [4]:
# Hough 直线检测 - 写 FPS 数值
def process_frame(img_bgr):
    
    # 记录该帧开始处理的时间
    start_time = time.time()
    
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY) # BGR 转 灰度图
    edges = cv2.Canny(img_gray, 50, 150, apertureSize=3) # Canny 直线检测
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 150)   # 霍夫直线检测
    
    if lines is None: # 如果检测出直线
        pass
    else:
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            # x1 stores the rounded off value of (r * cos(theta) - 1000 * sin(theta))
            x1 = int(x0 + 1000 * (-b))
            # y1 stores the rounded off value of (r * sin(theta)+ 1000 * cos(theta))
            y1 = int(y0 + 1000 * (a))
            # x2 stores the rounded off value of (r * cos(theta)+ 1000 * sin(theta))
            x2 = int(x0 - 1000 * (-b))
            # y2 stores the rounded off value of (r * sin(theta)- 1000 * cos(theta))
            y2 = int(y0 - 1000 * (a))
            img_bgr = cv2.line(img_bgr, (x1, y1), (x2, y2), (0, 0, 255), 2)
    
    # 在画面上写 FPS 数值
    end_time = time.time()
    FPS = 1/(end_time - start_time) # 计算每秒处理图像帧数FPS
    FPS_string = 'FPS {:.2f}'.format(FPS) # 写在画面上的字符串
    img_bgr = cv2.putText(img_bgr, FPS_string, (25, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (255, 0, 255), 2) # 在画面上写字：图片，字符串，左上角坐标，字体，字体大小，颜色，字体粗细
    
    return img_bgr

## 逐帧实时处理手机摄像头拍摄的画面-代码模板

In [ ]:
while True:
    img_bgr = cap.read()
    
    if img_bgr is None:
        continue
        
    img_bgr = process_frame(img_bgr)
    
    cvs.imshow(img_bgr)

## 点击`重启kernel`关闭摄像头